# BAN 612 - Project - Data Collection Part
Submitted by - Alarmelu Pichu Mani 
Net ID - TJ6723

The BAN612_webscraping_zillow_AlarmeluPM notebook is responsible for webscraping the Zillow real estate website for housing information in the San Ramon area in California.

In [59]:
# importing all the required libraries
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen
import pandas as pd
import requests
import re
import time
import math

The main_urls refer the to the search page we see when we open zillow and type in our area preference and other seacrh prefereces like price,
to see sold houses or for-sale houses and so on.

For the purposes of data collection and further analysis, I have searched for the most recently sold houses in San Ramon area

In [58]:
main_urls=[]
for i in range(2,6):
    url = 'https://www.zillow.com/san-ramon-ca/sold/'+str(i)+'_p/'
    main_urls.append(url)

Iterate through the main urls using selenium webdriver to get the individual house urls to collect more information

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import time
hyperlinks=[]
for url in main_urls:
    ## Selenium webdrive to mimic human behavior of opening urls
    options = Options()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36")
    driver = webdriver.Chrome('/Users/ushapraveen/Downloads/chromedriver',chrome_options=options)
    driver.get(url)
    
    # suspend execution for 10 seconds before opening the next url
    time.sleep(10) 
    
    #get the soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #close the browser tabs opened by selenium webdriver
    driver.quit()
    
    #find all hyperlinks in the soup 
    links = [a['href'] for a in soup.find_all('a', href=True)]
    print(links)
    
    # collect all the urls in that search page
    hyperlinks.append(links) 

In [60]:
#search and collect only the urls showing the individual houses
home_details=[]
for i in range(0,len(hyperlinks)):
    for j in hyperlinks[i]:
        if(re.search('https://www.zillow.com/homedetails/',j)!= None):
            home_details.append(j)
            
#clean the results to get only the unique urls
unique_homes = set(home_details)

#print the number of unique records we can see from this search
print('The number of unique house details in this search','\n')
print(len(unique_homes))

The number of unique house details in this search 

157


In [47]:
# this set of links were created after an iteration of unique_homes below. 
#A webdriver exception halted the execution and failed to collect data from the last 30 odd links
#I couldn't handle it programmatically, hence the manual intervention
homedetails_excep=['https://www.zillow.com/homedetails/510-Teresa-Pl-San-Ramon-CA-94583/18448517_zpid/', 'https://www.zillow.com/homedetails/796-Lakemont-Pl-UNIT-6-San-Ramon-CA-94582/18450720_zpid/', 'https://www.zillow.com/homedetails/321-Kingsley-Pl-San-Ramon-CA-94583/18447609_zpid/', 'https://www.zillow.com/homedetails/339-Goldfield-Pl-San-Ramon-CA-94582/241642379_zpid/', 'https://www.zillow.com/homedetails/2053-Echo-Pl-San-Ramon-CA-94582/18455447_zpid/', 'https://www.zillow.com/homedetails/7141-Watsonia-Dr-San-Ramon-CA-94582/68043064_zpid/', 'https://www.zillow.com/homedetails/9688-Ashby-Way-San-Ramon-CA-94583/18444573_zpid/', 'https://www.zillow.com/homedetails/140-Eastridge-Dr-San-Ramon-CA-94582/18451528_zpid/', 'https://www.zillow.com/homedetails/3012-Sorrelwood-Dr-San-Ramon-CA-94582/18462037_zpid/', 'https://www.zillow.com/homedetails/511-Skyline-Dr-San-Ramon-CA-94583/18440435_zpid/', 'https://www.zillow.com/homedetails/3010-Millbridge-Dr-San-Ramon-CA-94583/18447086_zpid/', 'https://www.zillow.com/homedetails/470-Bollinger-Canyon-Ln-APT-183-San-Ramon-CA-94582/68042886_zpid/', 'https://www.zillow.com/homedetails/11-Marilyn-Pl-San-Ramon-CA-94583/18447381_zpid/', 'https://www.zillow.com/homedetails/3136-Montbretia-Way-San-Ramon-CA-94582/122008857_zpid/', 'https://www.zillow.com/homedetails/323-Chiltern-Ct-San-Ramon-CA-94582/89038831_zpid/', 'https://www.zillow.com/homedetails/3310-Ensenada-Dr-San-Ramon-CA-94583/18447897_zpid/', 'https://www.zillow.com/homedetails/9085-Alcosta-Blvd-APT-320-San-Ramon-CA-94583/18445732_zpid/', 'https://www.zillow.com/homedetails/321-Norris-Canyon-Ter-San-Ramon-CA-94583/18450054_zpid/', 'https://www.zillow.com/homedetails/616-Peaceful-Valley-Dr-San-Ramon-CA-94582/18446671_zpid/', 'https://www.zillow.com/homedetails/7640-Stoneleaf-Rd-San-Ramon-CA-94582/95430853_zpid/', 'https://www.zillow.com/homedetails/101-Stone-Pine-Ln-San-Ramon-CA-94583/18440239_zpid/', 'https://www.zillow.com/homedetails/404-Fernando-Ct-San-Ramon-CA-94583/18440805_zpid/', 'https://www.zillow.com/homedetails/302-Eastridge-Dr-San-Ramon-CA-94582/18451452_zpid/', 'https://www.zillow.com/homedetails/124-Geranium-Ct-San-Ramon-CA-94582/68040269_zpid/', 'https://www.zillow.com/homedetails/122-Woodview-Cir-San-Ramon-CA-94582/18449786_zpid/', 'https://www.zillow.com/homedetails/9463-Thunderbird-Pl-San-Ramon-CA-94583/18445391_zpid/', 'https://www.zillow.com/homedetails/105-Reflections-Dr-APT-14-San-Ramon-CA-94583/72556641_zpid/', 'https://www.zillow.com/homedetails/1393-Bayberry-View-Ln-San-Ramon-CA-94582/122010096_zpid/', 'https://www.zillow.com/homedetails/106-Bandol-Ct-San-Ramon-CA-94582/68036912_zpid/', 'https://www.zillow.com/homedetails/2638-Derby-Dr-San-Ramon-CA-94583/18443069_zpid/', 'https://www.zillow.com/homedetails/516-La-Copita-Ct-San-Ramon-CA-94583/18440693_zpid/']

In [48]:
house_details_list=[] #list to collect individual rows of housing data
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
#for home in unique_homes: # this for-loop was the first data collection iteration that got interrupted by a webdriver exception
for home in homedetails_excep: #scraping the left out links using this iteration
    print(home)
    options = Options()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36")
    driver = webdriver.Chrome('/Users/ushapraveen/Downloads/chromedriver',chrome_options=options)
    driver.get(home)
    time.sleep(10)
    soup_page = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    #scraping the address
    address=[]
    try:
        temp_addr = soup_page.find('h1',attrs={'id':'ds-chip-property-address'}).findAll('span')
        address = clean_address(temp_addr)
    except:
        address = 'N/A'

    #scraping the zestimate
    try:
        zestimate = soup_page.find('span',attrs={'class':'Text-c11n-8-33-0__aiai24-0 flSprY'}).text
    except:
        zestimate = 'N/A'

    #scraping the no. of baths
    try:
        for i in soup_page.find('div',{'class':'ds-bed-bath-living-area-header'}):
            for j in i.findAll('span',{'class':'sc-psQdR iYtUny'}):
                temp.append(j.find('span').text)
        baths = temp[1]
        #print(baths)
    except:
        baths = 'N/A'
    #scraping the built year
    details=[]
    try:
        for i in soup_page.findAll('div',{'class':'ds-expandable-card-section-default-padding'}):
            for j in i.findAll('ul',{'class':'ds-home-fact-list'}):
                for k in j.findAll('li',{'class':'ds-home-fact-list-item'}):
                    for l in k.findAll('span',{'class':'Text-c11n-8-33-0__aiai24-0 sc-qXHHN kGAgDv'}):
                        details.append(k.text)
        year_built = details[1].strip('Year built:')
    except:
        year_built='N/A'
    #scraping thr sqft of the home
    try:
        sqft_soup = soup_page.findAll('div',attrs = {'class':'ds-bed-bath-living-area-header'})
        for i in sqft_soup:
            span = i.text
        sqft = span.replace(span[:8],'')
    except:
        sqft = 'N/A'

    #scraping the no. of bathrooms
    try:
        temp=[]
        for i in soup_page.find('div',{'class':'ds-bed-bath-living-area-header'}):
            for j in i.findAll('span',{'class':'sc-psQdR iYtUny'}):
                temp.append(j.find('span').text)
        baths = temp[1]
    except:
        baths='N/A'
        
    #scraping the school ratings
    try:
        ratings=[]
        for i in soup_page.findAll('ul',attrs={'id':'ds-nearby-schools-list'}):
            for j in i.findAll('span',attrs={'class':'Text-c11n-8-33-0__aiai24-0 flSprY'}):
                ratings.append(j.text)
        average_rating = avg_rating(ratings)
    except:
        average_rating ='N/A'
        
    #scraping the status and price of the home
    try:
        status_price = soup_page.find('span',attrs={'class':'sc-pItiW hckGML ds-status-details'}).text
    except:
        status_price = 'N/A'

    #Scrape other important details like bed,bath,materials used,appliances
    overview=[]
    try:
        for i in soup_page.findAll('li'):
            for j in i.findAll('ul',attrs={'class':'sc-pdjNk gCNoOc'}):
                overview.append(j.find('span').text)
        home_features = overview
    except:
        home_features = 'N/A'
    overview_list = clean_overview(overview)
    
    #scraping the HOA information
    hoa_temp=[]
    for i in soup_page.findAll('span',attrs={'class':'Text-c11n-8-33-0__aiai24-0 kTUCqv'}):
        hoa_temp.append(i.text)
    try:       
        for j in hoa_temp:
            if re.search('HOA fee:',j)!=None:
                hoa = j
    except:
        hoa = 'N/A'
        
    details_dict ={'Address':address,'Year built':year_built,'Zestimate':zestimate,'Baths':baths,'Sqft':sqft,'School_rating':average_rating,'Status':status_price,'Beds':overview_list[0],'Heating':overview_list[1],'Cooling':overview_list[2],'HomeType':overview_list[3],'Region':overview_list[4],'Hoa':hoa,'TaxAssessedValue':overview_list[6]}
    house_details_list.append(details_dict)

https://www.zillow.com/homedetails/510-Teresa-Pl-San-Ramon-CA-94583/18448517_zpid/


<ipython-input-48-2fc80b8f3cfb>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/Users/ushapraveen/Downloads/chromedriver',chrome_options=options)


https://www.zillow.com/homedetails/796-Lakemont-Pl-UNIT-6-San-Ramon-CA-94582/18450720_zpid/
https://www.zillow.com/homedetails/321-Kingsley-Pl-San-Ramon-CA-94583/18447609_zpid/
https://www.zillow.com/homedetails/339-Goldfield-Pl-San-Ramon-CA-94582/241642379_zpid/
https://www.zillow.com/homedetails/2053-Echo-Pl-San-Ramon-CA-94582/18455447_zpid/
https://www.zillow.com/homedetails/7141-Watsonia-Dr-San-Ramon-CA-94582/68043064_zpid/
https://www.zillow.com/homedetails/9688-Ashby-Way-San-Ramon-CA-94583/18444573_zpid/
https://www.zillow.com/homedetails/140-Eastridge-Dr-San-Ramon-CA-94582/18451528_zpid/
https://www.zillow.com/homedetails/3012-Sorrelwood-Dr-San-Ramon-CA-94582/18462037_zpid/
https://www.zillow.com/homedetails/511-Skyline-Dr-San-Ramon-CA-94583/18440435_zpid/
https://www.zillow.com/homedetails/3010-Millbridge-Dr-San-Ramon-CA-94583/18447086_zpid/
https://www.zillow.com/homedetails/470-Bollinger-Canyon-Ln-APT-183-San-Ramon-CA-94582/68042886_zpid/
https://www.zillow.com/homedetails/11

In [40]:
#output of the first iteration of unique_homes list 
#this gave about 126 records of houses in Sanramon
#df_data = pd.DataFrame(house_details_list)
df_data

,Address,Year built,Zestimate,Baths,Sqft,School_rating,Status,Beds,Heating,Cooling,HomeType,Region,Hoa,TaxAssessedValue
0,"22 Palamos Ct, <!-- -->San Ramon, CA 94583",1980,"$1,375,678",3,"1,880 sqft",9,"Sold: $1,350,000",4,Forced Air,Central Air,SingleFamily,San Ramon,HOA fee: $41 monthly,"$699,387"
1,"625 Santander Dr, <!-- -->San Ramon, CA 94583",1979,"$1,685,817",3,"2,585 sqft",9,Coming soon: Mar 4.,4,"Forced air, Gas",Central,SingleFamily,San Ramon,Has HOA fee: Yes,"$551,464"
2,"3542 Cinnamon Ridge Rd, <!-- -->San Ramon, CA...",2016,"$1,722,071",5,"2,721 sqft",9,"Sold: $1,750,000",4,Zoned,Zoned,SingleFamily,San Ramon,HOA fee: $96 monthly,"$1,229,832"
3,"2795 Norris Canyon Rd, <!-- -->San Ramon, CA ...",1974,"$2,309,553",4,"3,389 sqft",8,"Sold: $2,305,000",4,Zoned,"Ceiling Fan(s), Zoned",SingleFamily,San Ramon,Has HOA fee: No,"$1,012,680"
4,"2791 Camino Venadillo, <!-- -->San Ramon, CA ...",1974,"$1,666,454",3,"2,518 sqft",8,"Sold: $1,650,000",4,"Forced Air, Natural Gas","Central Air, Whole House Fan, ENERGY STAR Qua...",SingleFamily,San Ramon,HOA fee: $117 quarterly,"$680,626"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,"7633 Stellaria Ln, <!-- -->San Ramon, CA 94582",2000,"$1,081,956",3,"1,462 sqft",9,"Sold: $1,075,000",3,Forced Air,"Ceiling Fan(s), Central Air",Townhouse,San Ramon,HOA fee: $277 monthly,"$875,913"
122,"3578 Sandalford Way, <!-- -->San Ramon, CA 94582",2004,"$1,511,554",3,"2,962 sqft",9,"Sold: $1,500,000",4,Zoned,Zoned,SingleFamily,San Ramon,Has HOA fee: No,"$1,156,907"
123,"6117 Yardley Ln, <!-- -->San Ramon, CA 94582",2006,"$1,217,016",3,"2,479 sqft",9,"Sold: $1,205,000",5,"Zoned, Natural Gas","Ceiling Fan(s), Zoned",Townhouse,San Ramon,HOA fee: $294 monthly,"$864,884"
124,"3647 Crow Canyon Rd, <!-- -->San Ramon, CA 94582",1987,"$754,822",3,"1,386 sqft",9,"Sold: $740,000",3,Forced Air,Central Air,Condo,San Ramon,HOA fee: $450 monthly,"$609,907"


In [64]:
#the df_excep collects the next 30 records missed out in the earlier iteration.
df_data2 = pd.DataFrame(house_details_list)
df_data2

,Address,Year built,Zestimate,Baths,Sqft,School_rating,Status,Beds,Heating,Cooling,HomeType,Region,Hoa,TaxAssessedValue
0,"510 Teresa Pl, <!-- -->San Ramon, CA 94583",1978,"$1,740,603",3,"2,750 sqft",8,"Sold: $1,720,000",5,"Zoned, Natural Gas",Zoned,SingleFamily,San Ramon,Has HOA fee: No,"$690,020"
1,"796 Lakemont Pl UNIT 6, <!-- -->San Ramon, CA...",1990,"$962,333",3,"1,727 sqft",9,"Sold: $950,000",2,Forced Air,Central Air,Townhouse,San Ramon,HOA fee: $582 monthly,"$805,000"
2,"321 Kingsley Pl, <!-- -->San Ramon, CA 94583",1971,"$1,194,027",2,"1,479 sqft",9,"Sold: $1,170,000",4,Forced air,,SingleFamily,San Ramon,HOA fee: $582 monthly,"$227,298"
3,"339 Goldfield Pl, <!-- -->San Ramon, CA 94582",2016,"$1,409,758",3,"1,996 sqft",9,"Sold: $1,400,000",3,Zoned,Zoned,SingleFamily,San Ramon,HOA fee: $99 monthly,"$1,063,000"
4,"2053 Echo Pl, <!-- -->San Ramon, CA 94582",1989,"$1,177,549",3,"2,097 sqft",9,"Sold: $1,151,000",3,"Forced air, Gas",Other,Townhouse,San Ramon,Has HOA fee: Yes,"$923,104"
5,"7141 Watsonia Dr, <!-- -->San Ramon, CA 94582",2006,"$1,942,718",4,"3,175 sqft",9,"Sold: $1,938,888",4,"Zoned, Fireplace(s)",Zoned,SingleFamily,San Ramon,HOA fee: $20 monthly,"$1,272,052"
6,"9688 Ashby Way, <!-- -->San Ramon, CA 94583",1965,"$1,142,434",2,"1,202 sqft",9,"Sold: $1,120,000",3,Forced air,Central,SingleFamily,San Ramon,HOA fee: $20 monthly,"$865,943"
7,"140 Eastridge Dr, <!-- -->San Ramon, CA 94582",1988,"$445,712",1,730 sqft,8,"Sold: $440,000",1,Forced Air,Central Air,Condo,San Ramon,HOA fee: $336 monthly,"$436,968"
8,"3012 Sorrelwood Dr, <!-- -->San Ramon, CA 94582",1998,"$1,906,316",5,"3,847 sqft",9,"Sold: $1,895,000",5,Zoned,Zoned,SingleFamily,San Ramon,HOA fee: $40 monthly,"$1,523,082"
9,"511 Skyline Dr, <!-- -->San Ramon, CA 94583",1996,"$635,967",2,995 sqft,8,"Sold: $630,000",2,,,Condo,San Ramon,Has HOA fee: Yes,"$264,911"


Combine df_data first(set of data scraped) and df_data2 (the data collected after encountering a web driver exception) in to the final data set

In [65]:
df_final = df_data2.append(df_data, ignore_index=True)

In [68]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Address           157 non-null    object
 1   Year built        157 non-null    object
 2   Zestimate         157 non-null    object
 3   Baths             157 non-null    object
 4   Sqft              157 non-null    object
 5   School_rating     157 non-null    int64 
 6   Status            157 non-null    object
 7   Beds              157 non-null    object
 8   Heating           157 non-null    object
 9   Cooling           157 non-null    object
 10  HomeType          157 non-null    object
 11  Region            157 non-null    object
 12  Hoa               157 non-null    object
 13  TaxAssessedValue  157 non-null    object
dtypes: int64(1), object(13)
memory usage: 17.3+ KB


In [73]:
#clean the address column of some tag characters
df_final['Address'] = df_final.Address.str.replace('<!-- -->' , '')
df_final

,Address,Year built,Zestimate,Baths,Sqft,School_rating,Status,Beds,Heating,Cooling,HomeType,Region,Hoa,TaxAssessedValue
0,"510 Teresa Pl, San Ramon, CA 94583",1978,"$1,740,603",3,"2,750 sqft",8,"Sold: $1,720,000",5,"Zoned, Natural Gas",Zoned,SingleFamily,San Ramon,Has HOA fee: No,"$690,020"
1,"796 Lakemont Pl UNIT 6, San Ramon, CA 94582",1990,"$962,333",3,"1,727 sqft",9,"Sold: $950,000",2,Forced Air,Central Air,Townhouse,San Ramon,HOA fee: $582 monthly,"$805,000"
2,"321 Kingsley Pl, San Ramon, CA 94583",1971,"$1,194,027",2,"1,479 sqft",9,"Sold: $1,170,000",4,Forced air,,SingleFamily,San Ramon,HOA fee: $582 monthly,"$227,298"
3,"339 Goldfield Pl, San Ramon, CA 94582",2016,"$1,409,758",3,"1,996 sqft",9,"Sold: $1,400,000",3,Zoned,Zoned,SingleFamily,San Ramon,HOA fee: $99 monthly,"$1,063,000"
4,"2053 Echo Pl, San Ramon, CA 94582",1989,"$1,177,549",3,"2,097 sqft",9,"Sold: $1,151,000",3,"Forced air, Gas",Other,Townhouse,San Ramon,Has HOA fee: Yes,"$923,104"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"7633 Stellaria Ln, San Ramon, CA 94582",2000,"$1,081,956",3,"1,462 sqft",9,"Sold: $1,075,000",3,Forced Air,"Ceiling Fan(s), Central Air",Townhouse,San Ramon,HOA fee: $277 monthly,"$875,913"
153,"3578 Sandalford Way, San Ramon, CA 94582",2004,"$1,511,554",3,"2,962 sqft",9,"Sold: $1,500,000",4,Zoned,Zoned,SingleFamily,San Ramon,Has HOA fee: No,"$1,156,907"
154,"6117 Yardley Ln, San Ramon, CA 94582",2006,"$1,217,016",3,"2,479 sqft",9,"Sold: $1,205,000",5,"Zoned, Natural Gas","Ceiling Fan(s), Zoned",Townhouse,San Ramon,HOA fee: $294 monthly,"$864,884"
155,"3647 Crow Canyon Rd, San Ramon, CA 94582",1987,"$754,822",3,"1,386 sqft",9,"Sold: $740,000",3,Forced Air,Central Air,Condo,San Ramon,HOA fee: $450 monthly,"$609,907"


Capture all the extracted data into a csv file - SanRamon_housing.csv

In [74]:
#convert the df_final dataframe to a csv file
df_final.to_csv('SanRamon_housing.csv')

In [75]:
############# END OF DATA SCRAPING AND EXTRACTION ############# 

# Below are some of the utility functions used while scraping and data extraction

clean_address function cleans the address obtained from the tags and extracts only the Address value between the span tags

In [32]:
import re
def clean_address(address):
    clean_address=" "
    pattern = re.compile('<span>?(.+?)</span>')
    for i in address:
        temp = re.findall(pattern,str(i))
        clean_address+=temp[0]
    return clean_address

The avg_rating function takes in the school ratings as a list of strings, converts them to integers and calculates the average

In [33]:
def avg_rating(ratings):
    sum_rating = 0
    for i in ratings:
        sum_rating+=int(i)
    avg_rating = sum_rating/(len(ratings))
    return round(avg_rating)

The clean_overview function cleans the information extracted from the home features overview section of each home's page.

In [34]:
def clean_overview(overview):
    overview_list = ['','','','','','No','']
    for i in overview:
        temp = i.split(':')
        if temp[0]=='Bedrooms':
            overview_list.insert(0,temp[1])
        if temp[0] == 'Heating features':
            overview_list.insert(1,temp[1])
        if temp[0] == 'Cooling features':
            overview_list.insert(2,temp[1])
        if temp[0] == 'Home type':
            overview_list.insert(3,temp[1])
        if temp[0] == 'Region':
            overview_list.insert(4,temp[1])
        if temp[0] == 'Tax assessed value':
            overview_list.insert(6,temp[1])        
    return overview_list       